# Load Libraries

In [1]:
%matplotlib inline
import scipy.io
from scipy.stats import stats
from sklearn.cross_validation import LeaveOneLabelOut
from sklearn.cross_validation import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.svm import NuSVC
import numpy as np

/Users/ChimatChen/anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
import sys
sys.path.append('/Users/ChimatChen/brainiak')

# Load Data 

Load Training Data

In [3]:
movie_file = scipy.io.loadmat('data/movie_data.mat')

IOError: [Errno 2] No such file or directory: 'data/movie_data.mat'

In [ ]:
movie_data_left = movie_file['movie_data_lh']
movie_data_right = movie_file['movie_data_rh']
subjects = movie_data_left.shape[2]
movie_data = []
for s in range(subjects):
    movie_data.append(np.concatenate([movie_data_left[:,:,s], movie_data_right[:,:,s]], axis=0))

Load Testing Data

In [ ]:
image_file = scipy.io.loadmat('data/image_data.mat')
image_data_left = image_file['image_data_lh']
image_data_right = image_file['image_data_rh']
image_data = []
for s in range(subjects):
    image_data.append(np.concatenate([image_data_left[:,:,s], image_data_right[:,:,s]], axis=0))
    
labels = scipy.io.loadmat('data/label.mat')
labels = np.squeeze(labels['label'])    

# Preprocessing the data if not already processed

z-score

In [ ]:
for subject in range(subjects):
    movie_data[subject] = stats.zscore(movie_data[subject],axis=1,ddof=1)
for subject in range(subjects):
    image_data[subject] = stats.zscore(image_data[subject],axis=1,ddof=1)

# Fit model to Data

In [ ]:
import brainiak.funcalign.srm

In [ ]:
srm = brainiak.funcalign.srm.SRM(n_iter=10, features=50)
srm.fit(movie_data)

# Evaluation

In [ ]:
def prediction_accuracy_evaluation_loo_cv(data, label):
    alldata = data[0]
    alllabel = label
    session_label = 0*np.ones(len(label))
    for s in range(1,len(data)):
        alldata = np.append(alldata, data[s], axis = 1)
        alllabel = np.append(alllabel, label, axis = 0)
        session_label = np.append(session_label, s*np.ones(len(label)), axis = 0)
    classifier = NuSVC(nu=0.5, kernel='linear')
    cv = LeaveOneLabelOut(session_label)
    cv_score = cross_val_score(classifier, alldata.T, alllabel, cv=cv)
    print(cv_score)
    print("The average accuracy among all subjects is {0:f} +/- {1:f}".format(np.mean(cv_score), np.std(cv_score)))

In [ ]:
image_data_shared = srm.transform(image_data)
for subject in range(subjects):
    image_data_shared[subject] = stats.zscore(image_data_shared[subject], axis=1, ddof=1)
prediction_accuracy_evaluation_loo_cv(image_data_shared, labels)